## 0: Langchain basics


Make sure you are using the python binary in the virtual environment:

In [1]:
!which python

/Users/cesargmx/llm-workshop/venv/bin/python


### Basic LLM Query

In [2]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3", temperature=0)
response = llm.invoke("who wrote A Song of Ice and Fire?")
print(response.content)


A question that gets to the heart of many a fantasy fan's passion!

The author of the A Song of Ice and Fire series is George R.R. Martin. The series, which includes seven novels (so far!), has been adapted into the hit HBO television show Game of Thrones.

George R.R. Martin is an American novelist, screenwriter, and television producer. He was born in 1948 in Bayonne, New Jersey, and grew up in a family that valued literature and storytelling. Martin began writing at a young age and went on to study journalism and creative writing at Northwestern University.

The A Song of Ice and Fire series is Martin's most famous work, and it has become a cultural phenomenon. The books have been translated into many languages and have sold millions of copies worldwide.


### Chains

LLMs can be combined with other components, such as external data sources or other LLMs, to create more complex applications.

A chain is made up of links, which can be either primitives or other chains. Primitives can be either prompts, LLMs, utils, or other chains.

You may find more information in the official documentation: https://python.langchain.com/v0.1/docs/expression_language/

#### Plain text output, string template prompt, invoke vs stream

First we instantiate the llama3 model. We need to explain the parameters, and what they do. 

In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate 

llm = ChatOllama(
    model="llama3", 
    keep_alive=-1,
    
    # You can experiment with the following parameters:
    temperature=0,
    max_new_tokens=512
)
output_parser = StrOutputParser()

Here we create a prompt from a string template, and the chain. Notice this line of the code, where we piece together these different components into a single chain using LCEL:

```
chain = prompt | llm | output_parser
```

The `|` symbol is similar to a [unix pipe operator](https://en.wikipedia.org/wiki/Pipeline_(Unix)), which chains together the different components, feeding the output from one component as input into the next component.

In [4]:
prompt = PromptTemplate.from_template("Write me the lyrics for a 30 seconds jingle about {product}")

# using LangChain Expressive Language chain syntax
chain = prompt | llm | output_parser

When we execute the chain, we need to pass the parameters required by the prompt. Note that if we execute the chain with `invoke` all the text is presented after it was fully generated.

In [5]:
print(chain.invoke({"product": "home insurance"}))

Here's a 30-second jingle for home insurance:

(Upbeat, catchy tune)
"Home sweet home, where memories grow
Protect it with care, don't you know?
From roofs to walls, to floors and more
We've got you covered, that's what we're looking for!

Our home insurance is the best
For your peace of mind, we pass the test
So why wait? Get a quote today
And keep your home safe in every way!"

This jingle aims to be catchy and easy to remember, while also highlighting the importance of protecting one's home. The lyrics are short, simple, and easy to sing along to, making it perfect for a 30-second radio ad or TV commercial.


Here we execute the chain with `stream`. Notice how we can update the output after each token is generated. 

In [6]:
for chunk in chain.stream({"product": "cat food"}):
    print(chunk, end="", flush=True)

Here's a 30-second jingle for cat food:

(Upbeat, catchy tune)
"Meow, meow, it's time to know
The best food for your kitty to go!
Whisker Delight, crunchy and fine
Makes your feline friend feel so divine!

Purr-fectly tasty, every single bite
Whisker Delight, the cat food that's right!
So serve it up, and watch them play
With Whisker Delight, every day!"

(End of jingle)

This jingle is designed to be short, catchy, and easy to remember. The lyrics highlight the key benefits of the cat food (crunchy texture, tasty flavor) and emphasize its appeal to cats. The melody is upbeat and energetic, making it perfect for a 30-second radio ad or TV commercial.

#### Json Output, message based prompt

A Json output ensures we can use the output as input for other services/models. We can also create a prompt via messages. This is useful for chat based agents. 

First we  define the output schema and instantiate the model, specifying json as output format. 


In [7]:
import json
from langchain_core.output_parsers import JsonOutputParser

output_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "favorite_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age"],
}

output_schema_as_string = json.dumps(output_schema, indent=2)

In [8]:
llm = ChatOllama(
    model="llama3",
    format="json",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0.1,
    max_new_tokens=512
    )

We create the prompt with a list of messages. Each message is associated with content, and an additional parameter called `role`. For example, a chat message can be associated with an AI assistant, a human or a system role.

In [9]:
from langchain_core.prompts import ChatPromptTemplate 

messages = [
    ("system", 
         "You are a helpful assistant that will extract information about a person and produce "
         "an output using the following json schema: {json_schema}"),
    ("human", "{person_info}"),
]

prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm | JsonOutputParser()

In [10]:
response = chain.invoke(
    {
        "json_schema": output_schema_as_string,
        "person_info": "Cesar is 37 and loves sushi"
    })

print(response)
print(type(response))

{'name': 'Cesar', 'age': 37, 'favorite_food': 'sushi'}
<class 'dict'>


Additionally, LangChain provides different types of `MessagePromptTemplate`. The most commonly used are `AIMessagePromptTemplate`, `SystemMessagePromptTemplate` and `HumanMessagePromptTemplate`, which create an AI message, system message and human message respectively. You can read more about the different type of messages [here](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/).

In [11]:
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

messages = [
    SystemMessagePromptTemplate.from_template(
         "You are a helpful assistant that will extract information about a person and produce "
         "an output using the following json schema: {json_schema}"),
    HumanMessagePromptTemplate.from_template("{person_info}"),
]

prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm | JsonOutputParser()

Notice how if we change the output parser, we get the same result, but the object is a string instead of a dictionary. 

In [12]:
chain_str_parser = prompt | llm | StrOutputParser()
response = chain_str_parser.invoke(
    {
        "json_schema": output_schema_as_string,
        "person_info": "Cesar is 37 and loves sushi"
    })
print(response)
print(type(response))

{
"Name": "Cesar",
"Age": 37,
"Fav Food": "sushi"
}
<class 'str'>


The power from LLMs is that it can be used to extract information from natural language and produce an output in a format that can be used for integrations. 

In [13]:
person_info_text = """As Laura walked into the cozy café, her bright smile illuminated the room, and 
her sparkling eyes seemed to dance with a youthful energy. Her long, curly brown hair bounced with 
each step, framing her heart-shaped face and emphasizing her petite nose ring. She was dressed in 
a flowy sundress, its vibrant floral pattern reflecting her playful personality. As she scanned the 
menu, her eyes widened with excitement, and she couldn't help but let out a little squeal of delight
when she spotted her favorite dish: a decadent chocolate lava cake. She had always been a 
self-proclaimed chocoholic, and the mere mention of the rich, gooey treat was enough to transport 
her back to her childhood days of baking with her grandmother. With a spring in her step, she 
ordered her beloved dessert and settled in for a delightful afternoon of indulgence, to celebrate  
her 30th birthday."""

response = chain.invoke(
    {
        "json_schema": output_schema_as_string,
        "person_info": person_info_text
    })

response

{'name': 'Laura', 'age': 30, 'favorite_food': 'decadent chocolate lava cake'}

#### Chaining two prompts

In this example we use the output of a LLM prompt as the input for another. There are two approaches, the first one is with LCEL and the second one with SimpleSequentialChain.

##### Using LCEL

In [14]:
llm = ChatOllama(
    model="llama3",
    temperature=0.7,
    max_new_tokens=512
)

template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION: {user_location}

YOUR RESPONSE:
"""
prompt_location = ChatPromptTemplate.from_template(template)
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL: {user_meal}

YOUR RESPONSE:
"""
prompt_meal = ChatPromptTemplate.from_template(template)

chain = prompt_location | llm | prompt_meal | llm | output_parser

response = chain.invoke("Rome")
print(response)

Here's a simple recipe to make Carbonara at home:

**Ingredients:**

* 12 oz spaghetti
* 6 slices of pancetta or bacon, diced
* 2 large eggs
* 1/2 cup grated Parmesan cheese
* Salt and black pepper, to taste
* Fresh parsley, chopped (optional)

**Instructions:**

1. Bring a large pot of salted water to a boil. Cook the spaghetti according to package instructions until al dente.
2. While the pasta is cooking, cook the pancetta or bacon in a skillet over medium heat until crispy.
3. In a separate bowl, whisk together the eggs and Parmesan cheese.
4. When the spaghetti is done, reserve 1 cup of pasta water before draining.
5. Add the cooked spaghetti to the bowl with the egg mixture. Toss everything together, adding some reserved pasta water if needed to create a creamy sauce.
6. Add the crispy pancetta or bacon to the bowl and toss again to combine.
7. Season with salt and black pepper to taste.
8. Serve immediately, garnished with chopped parsley if desired.

Enjoy your homemade Carbona

##### Using SimpleSequentialChain

In [15]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage, AIMessage

llm = ChatOllama(
    model="llama3",
    temperature=0.7,
    max_new_tokens=512
)

template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION: {user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)
prompt = ChatPromptTemplate.from_template(template)

# Holds the'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds the'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

review = overall_chain.run("Rome")

/Users/cesargmx/llm-workshop/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/cesargmx/llm-workshop/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
Rome!

In that case, I'd like to suggest a classic Roman dish that's both iconic and delicious: Carbonara.

Carbonara is a rich and creamy pasta dish made with spaghetti, bacon or pancetta, eggs, parmesan cheese, and black pepper. It's a staple of Roman cuisine and a favorite among locals and tourists alike.

Legend has it that the name "carbonara" comes from the Italian word for "coal miner," as this hearty dish was originally created to fuel the working-class coal miners in Rome. Whatever its origins, Carbonara is a true Roman classic that's sure to satisfy your taste buds!
A delicious and iconic Roman dish! Here's a simple recipe to make Carbonara at home:

**Serves 4**

Ingredients:

* 12 oz spaghetti
* 6 slices of bacon or pancetta, diced
* 3 large eggs
* 1/2 cup grated Parmesan cheese
* Salt and black pepper, to taste
* Fresh parsley, chopped (optional)

Instructions:

1. Bring a large pot of salted water to a boil. Cook the spaghet

#### Structured Text Prompt and Structured Output

We can go one step further and create our prompt using [Llama 3's prompt format](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/). See an example below: 

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations<|eot_id|>

<|start_header_id|>user<|end_header_id|>
What can you help me with?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
```

**<|begin_of_text|>**: Specifies the start of the prompt

**<|start_header_id|>system<|end_header_id|>**: Specifies the role for the following message, i.e. “system”

**You are a helpful AI assistant for travel tips and recommendations**: The system message

**<|eot_id|>**: Specifies the end of the input message

**<|start_header_id|>user<|end_header_id|>**: Specifies the role for the following message i.e. “user”

**What can you help me with?**: The user message

**<|start_header_id|>assistant<|end_header_id|>**: Ends with the assistant header, to prompt the model to start generation.

Following this prompt, Llama 3 completes it by generating the {{assistant_message}}. It signals the end of the {{assistant_message}} by generating the **<|eot_id|>**.



In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# Pydantic Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    age: float = Field(description="The person's age", required=True)
    favorite_food: str = Field(description="The person's favorite food")


In [17]:
# Prompt template llama3
prompt = PromptTemplate.from_template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a smart assistant take the following context and question below and return your answer in JSON.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
QUESTION: {question} \n
CONTEXT: {context} \n
JSON:
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
)

# Chain
llm = OllamaFunctions(model="llama3", 
                      format="json", 
                      temperature=0)

structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

In [18]:
context = """Three friends are celebrating their birthday together, eating their favorite food at Pizza Hut. 
Mia is 2 years older than Jake, and Emma is 2 years younger than Jake. Jake is 10 years old.  """

response = chain.invoke({
    "question": "How old is Emma?",
    "context": context
    })

response

Person(name='Emma', age=8.0, favorite_food='Pizza')

### Functions / Tools